In [1]:
import torch

# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
# check if cuda is available
print(torch.cuda.is_available())

True
True
False


/Users/rylanspence/opt/anaconda3/envs/diffusion/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import h5py
import numpy as np
import time
import timeit
import os

# import filters
import Estkf,Ensrf,Senkf
import Estkf_torch,Ensrf_torch,Senkf_torch



In [17]:
def stats(x):
    return x.shape, np.mean(x,axis=1), np.cov(x[:1000,:],ddof=1)

def close(state_analysis,xb):
    return np.allclose(state_analysis.mean(axis=1),xb.mean(axis=1))

def timer(filter,filter_object):
    """
    function execution time
    """
    st = time.time()
    x = filter_object._assimilate()
    et = time.time()

    # get the execution time
    elapsed_time = et - st
    
    return f"Filter: {filter.__name__} Execution time: {elapsed_time} seconds"


def timeit(filter,filter_object):
    """
    function execution time using python timeit module
    """
    %timeit filter_object._assimilate()
    


In [4]:
hf = h5py.File('../data/example_sim_outputs.h5',"r")
zeta = np.array(hf.get('zeta'))
obs = np.array(hf.get('obs'))
    
print(f"Ensemble state matrix dimensions (ne x nx): {zeta.shape}")
print(f"Observation vector at one time step dimensions: (ny) {obs.shape}")


state_forecast = zeta.T[:,:5]                            # ensemble of state vectors
observations = obs                                      # model observations (adcirc output file fort.63.nc)                  
model_observations = np.eye(obs.size)[:,:5]             # observation operator ( adcirc output file fort.61.nc)

params = [state_forecast,model_observations,observations]

Ensemble state matrix dimensions (ne x nx): (10, 3352598)
Observation vector at one time step dimensions: (ny) (11,)


In [21]:
filters = [Estkf,Ensrf,Senkf]

for i, filter in enumerate(filters):
    filter_class= getattr(filter,filter.__name__.lower())
    filter_object= filter_class(*params)
    print(f"{timer(filter,filter_object)}")
    timeit(filter,filter_object)
    print("\n")
   

Filter: Estkf Execution time: 0.07293105125427246 seconds
63.4 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Filter: Ensrf Execution time: 0.27252817153930664 seconds
255 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Filter: Senkf Execution time: 0.055776119232177734 seconds
55.7 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)




In [6]:
ensrf =  Ensrf.ensrf(*params)
ensrf_torch =  Ensrf_torch.ensrf_torch(*params)



In [7]:
%timeit ensrf._assimilate()

260 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
ensrf_torch._assimilate()

array([[0.3049049 , 0.30490485, 0.30490488, 0.3049049 , 0.3049049 ],
       [0.30509534, 0.3051078 , 0.30427524, 0.30740955, 0.30782387],
       [0.3053599 , 0.30538484, 0.30483666, 0.3106181 , 0.31072247],
       ...,
       [0.3063546 , 0.29884186, 0.3137718 , 0.30100474, 0.34468943],
       [0.30635422, 0.29884523, 0.31376734, 0.30100557, 0.34468526],
       [0.30635437, 0.2988422 , 0.31376666, 0.3010125 , 0.34467775]],
      dtype=float32)

In [9]:
# %timeit ensrf_torch._assimilate()

In [10]:
# print(stats(x1),"\n")
# print(stats(x2),"\n")
# print(close(x1,x2),"\n")
